# L1 and L2 Regularization - Elastic Net

Regularizarea de tipul Elastic Net combină cele două tipuri de regularizări (Ridge și Lasso). Prin urmare o să ia în considerare suma valorilor pătrate ale coeficiențior împreună cu un parametru (termen de penalizare) Lambda (lambda 1) și suma valorilor absolute ale coeficienților cu un alt parametru de penalizare Lambda 2. În cadrul acestui tip de Elastic Net Regression o să avem parte de două valori lambda. Există posibilitatea de a rescrie ecuația astfel încât să se folosească doar un parametrul de lambda împreună cu un parametrul de alpha care să reprezinte rația dintre Lasso Regression ți Ridge Regression. Cea de a doua variantă este mai practică deoarece ne permite să setăm o singură valoare (alpha) și astfel se setează valori pentru ambii parametri de lambda

În cadrul Scikit-Learn folosirea unui Elastic Net Regression este asemănătoare cu folosirea Ridge Regression sau Lasso Regression. Să începem să pregătim datele pentru Elastic Net

In [1]:
# importing the libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# reading the data into a DataFrame
df = pd.read_csv('../data/08-Linear-Regression-Models/Advertising.csv')

In [3]:
# printing the head of the DataFrame
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
# splitting the data into features and labels
X = df.drop('sales', axis=1)
y = df['sales']

In [5]:
# importing the Polynomial Converter
from sklearn.preprocessing import PolynomialFeatures

In [6]:
# create an instance of the PolynomialFeatures converter
polynomial_converter = PolynomialFeatures(degree=3, include_bias=False)

In [7]:
# fit and transform the data
polynomial_features = polynomial_converter.fit_transform(X)

In [8]:
# importing the train-test split method
from sklearn.model_selection import train_test_split

In [9]:
# splitting the data into train-test sets
X_train, X_test, y_train, y_test = train_test_split(polynomial_features, y, test_size=0.3, random_state=101)

In [10]:
# import the Standard Scaler 
from sklearn.preprocessing import StandardScaler

In [11]:
# create an instance of the StandardScaler
scaler = StandardScaler()

In [13]:
# fit the data to the scaler
scaler.fit(X_train)

StandardScaler()

In [14]:
# transform the data with the scaler
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

După ce am pregătit datele (crearea de polynomial features și standardizarea datelor) putem să începem să implementăm modelul respectiv. Pentru a putea utiliza modelul (o să începem direct cu modelul care realizează și cross-valdation) trebuie să importăm ElasticNetCV din sklearn.linear_model

In [15]:
# importing the Elastic Net Regression model
from sklearn.linear_model import ElasticNetCV

In [16]:
help(ElasticNetCV)

Help on class ElasticNetCV in module sklearn.linear_model._coordinate_descent:

class ElasticNetCV(sklearn.base.RegressorMixin, LinearModelCV)
 |  ElasticNetCV(*, l1_ratio=0.5, eps=0.001, n_alphas=100, alphas=None, fit_intercept=True, normalize='deprecated', precompute='auto', max_iter=1000, tol=0.0001, cv=None, copy_X=True, verbose=0, n_jobs=None, positive=False, random_state=None, selection='cyclic')
 |  
 |  Elastic Net model with iterative fitting along a regularization path.
 |  
 |  See glossary entry for :term:`cross-validation estimator`.
 |  
 |  Read more in the :ref:`User Guide <elastic_net>`.
 |  
 |  Parameters
 |  ----------
 |  l1_ratio : float or list of float, default=0.5
 |      Float between 0 and 1 passed to ElasticNet (scaling between
 |      l1 and l2 penalties). For ``l1_ratio = 0``
 |      the penalty is an L2 penalty. For ``l1_ratio = 1`` it is an L1 penalty.
 |      For ``0 < l1_ratio < 1``, the penalty is a combination of L1 and L2
 |      This parameter can 

Documentația de la ElasticNetCV ne spune că avem din nou parametrii de eps și n_alphas prin care putem să selectăm valoarea pentru parametrul lambda (pentru primul parametru lambda). După cum spuneam, mai există un alt parametru de lambda care însă este trecut ca și o rație între cele două valori. Acea rație se setează cu parametrul l1_ratio. Acestui parametru o să îi oferim ca și valori o listă de valori, iar documentația ne spune că atunci cu cât valoarea este mai apropiată de 1 se va folosi mai mult Lasso Regression (Lasso Regression o să aibă o rație mai mare în comparație cu Ridge Regression). Un model bun de valori care ar trebui oferit pentru acest parametru este [.1, .5, .7, .9, .95, .99, 1]

O să creem un model cu valorile recomandate pentru parametrii de l1_ration, eps și n_alphas (o să modificăm doar valoarea pentru max_iter pentru a nu avea acel warning)

In [28]:
elastic_cv_model = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], eps=0.001, n_alphas=100, max_iter=1_000_000)

In [29]:
elastic_cv_model.fit(X_train, y_train)

ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.95, 0.99, 1], max_iter=1000000)

După ce am antrenat modelul putem să vedem care sunt parametrii cei mai buni pentru acest model (l1_ratio_ și alpha_)

In [30]:
elastic_cv_model.alpha_

0.004943070909225827

In [31]:
elastic_cv_model.l1_ratio_

1.0

Din moment ce se folosește valoarea de 1.0 pentru rația dintre L1 regularization și L2 regularization, asta înseamnă că L1 regularization (Lasso Regression) are o valoare de 100%, prin urmare este un model care folosește doar Lasso Regression. Acest lucru este destul de comun pentru seturi mici de date. Putem să facem predicții și să evaluăm modelul.

In [32]:
# making predictions
y_pred = elastic_cv_model.predict(X_test)

In [33]:
# importing the error metrics
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [34]:
MAE = mean_squared_error(y_test, y_pred)

In [35]:
MAE

0.367616757419907

In [36]:
RMSE = np.sqrt(mean_squared_error(y_test, y_pred))

In [37]:
RMSE

0.6063140748984036

## Recapitulare

În cadrul acestui curs am învățat următoarele lucruri:

    1. Ce este Elastic Net Regularization

        Este o combinație între L1 regularization și L2 regularization
    
    2. De unde să importăm modelul de Elastic Net care se ocupă și cu partea de cross-validation

        from sklearn.linear_model import ElasticNetCV

    3. Cum să creem un model de Elastic Net Regression

        elastic_cv_model = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 0.99, 1], eps=0.001, n_alphas=100, max_iter=1_000_000)